# Development Notebook

### Imports

In [21]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [9]:
# -- private imports

In [92]:
# -- dev imports

%load_ext autoreload
%autoreload 2
from python_files.data import KaggleDataset, create_labels_doc_level
from python_files.run import train_longformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model Configuration

### Tokenizer

In [20]:
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-large')
tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-large', add_prefix_space=True)
# def need an encoding function to categoize the inputs or at least to release them, maybe need to store them together tbh?

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


### Model and Optmizer

In [49]:
config_model = AutoConfig.from_pretrained('google/bigbird-roberta-large') 
config_model.num_labels = 3 # might have to change if we're using the numbered discourse type as well
#config_model.save_pretrained('model')


model = AutoModelForTokenClassification.from_pretrained('google/bigbird-roberta-large', config=config_model)
optimizer = torch.optim.Adam(params=model.parameters())

Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were n

## Data Preparation

In [66]:
texts = [
            'Hi my name is Yousef Nami',
            'Hi I am Shirin'
        ]
labels = [
    ['O', 'O', 'O', 'O', 'B-PERS', 'I-PERS'],
    ['O', 'O', 'O', 'B-PERS']
]

# TODO see if bert can accept text inputs?
labels_numeric = [
    [0, 0, 0, 0, 1, 2],
    [0, 0, 0, 1]
]

In [67]:
df = pd.DataFrame({
        'text': texts,
        'labels': labels_numeric
    })

In [80]:
dataset = KaggleDataset(
    df,
    tokenizer,
    max_length=20
)

In [81]:
for (inputs, targets) in dataset:
    print(targets)
    

tensor([0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
"You sef Na mi" -> "BNAME, BNAME, INAME, INAME"

## Data Training

In [82]:
train_loader = DataLoader(dataset, shuffle=True, batch_size=2)

In [86]:
train_longformer(model=model, optimizer=optimizer , epochs=1, train_loader=train_loader)

CUDA device NOT detected. Using CPU...
EPOCH 1 STARTED
---------------
tensor(1.1120, grad_fn=<NllLossBackward0>)
tensor([[[-0.0224, -0.0946,  0.0991],
         [-0.0681, -0.0895,  0.0942],
         [ 0.0401, -0.0928, -0.0653],
         [-0.0004,  0.0033,  0.2081],
         [-0.1006, -0.1141,  0.1016],
         [-0.0174, -0.0251,  0.1580],
         [ 0.0569,  0.0183,  0.0796],
         [ 0.0120, -0.0319,  0.0621],
         [-0.0059, -0.1213,  0.1128],
         [-0.1177, -0.0320,  0.0397],
         [-0.1405, -0.0022,  0.0908],
         [ 0.0113, -0.0479,  0.1648],
         [ 0.0282, -0.1020,  0.0731],
         [-0.0592, -0.0598, -0.0445],
         [ 0.2501, -0.1613, -0.2957],
         [-0.0495, -0.1887,  0.0911],
         [ 0.0622, -0.1628,  0.0522],
         [-0.0403,  0.0131,  0.1195],
         [ 0.0737, -0.0593,  0.0306],
         [-0.0229,  0.0390,  0.1207]],

        [[-0.0111,  0.0077, -0.1385],
         [ 0.1553, -0.1344, -0.0256],
         [ 0.1078,  0.0514,  0.1017],
         [

Exception: 

## Test on Kaggle dataset

In [93]:
PATH = '../../data/kaggle/feedback-prize-2021/train'
FILE_PATH = '../../data/kaggle/feedback-prize-2021/train.csv'

df_texts = create_labels_doc_level(PATH, FILE_PATH)
#df_texts[['text', 'labels']].to_csv('../../../data/kaggle/df_cleaned.csv')

#df_kaggle = pd.read_csv('../../data/kaggle/df_cleaned.csv')
#df_kaggle.head()
df_texts.head()

,id,text,text_split,labels,range,labels_temp
0,3321A3E87AD3,I do agree that some students would benefit fr...,"[I, do, agree, that, some, students, would, be...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,DFEAEC512BAB,Should students design a summer project for sc...,"[Should, students, design, a, summer, project,...","[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, ...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[3, 4, 4, 4, 4, 4, 4, 7, 8, 8, 8, 8, 8, 8, 8, ..."
2,2E4AFCD3987F,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[Dear, State, Senator, ,, In, the, ruels, of, ...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, ...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, 8, 8, 8, 8, ..."
3,EB6C2AF20BFE,People sometimes have a different opinion than...,"[People, sometimes, have, a, different, opinio...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,A91A08E523D5,"Dear senator,\n\nAs you know the Electoral Col...","[Dear, senator,, As, you, know, the, Electoral...","[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [95]:
df_kaggle = df_texts[['text', 'labels']]
df_kaggle.head()

,text,labels
0,I do agree that some students would benefit fr...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,Should students design a summer project for sc...,"[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, ..."
2,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, ..."
3,People sometimes have a different opinion than...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,"Dear senator,\n\nAs you know the Electoral Col...","[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [96]:
kaggle_dataset = KaggleDataset(df_kaggle, tokenizer, max_length=100)

In [97]:
k_train_loader = DataLoader(kaggle_dataset, shuffle=True, batch_size=1)

In [ ]:
train_longformer(model=model, optimizer=optimizer, epochs=1, train_loader=k_train_loader)

EPOCH 1 STARTED
---------------



KeyboardInterrupt

